In [ ]:
import requests
import os
import json
import pandas as pd

from typing import Any, Hashable

import src.agent as agent

In [ ]:
df = pd.read_csv("data/data_faulty.csv")

In [ ]:
plan_str = agent.get_cleaning_plan(df[["equipment_type", "component"]], backend="ollama", model="qwen2.5:3b")
plan = json.loads(plan_str)
plan


In [ ]:
df2 = df.copy()
df2 = agent.apply_cleaning_plan(df2, plan)

In [ ]:
def validate_result(df_antes, df_depois, label):
    df_merged = pd.merge(df_antes[['id', label]], df_depois[['id', label]], on='id', how='inner', suffixes=('_antes', '_depois'))
    df_merged_modificado = df_merged[df_merged[f'{label}_antes'] != df_merged[f'{label}_depois']]
    df_merged_igual = df_merged[df_merged[f'{label}_antes'] == df_merged[f'{label}_depois']]

    print(f"""
    Orginalmente existiam: {df_antes.shape[0]} registros

    {df_merged_modificado.shape[0]} registros foram alterados
    {df_merged_igual.shape[0]} registros permaneceram inalterados
    """)
    print('### Alterados ########')
    print(df_merged_modificado[[f'{label}_antes', f'{label}_depois']].drop_duplicates())

    print('### Iguais ##########')
    print(df_merged_igual[[f'{label}_antes', f'{label}_depois']].drop_duplicates())


In [ ]:
validate_result(df, df2, 'equipment_type')

In [ ]:
validate_result(df, df2, 'component')